In [1]:
import pika
import pickle
import numpy as np
import json

In [2]:
with open('../myfile.pkl', 'rb') as pkl_file:
    regressor = pickle.load(pkl_file)

/home/greg/anaconda3/lib/python3.9/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator LinearRegression from version 0.23.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [3]:
connection = pika.BlockingConnection(pika.ConnectionParameters(host='localhost'))
channel = connection.channel()

In [4]:
channel.queue_declare(queue='Features')

<METHOD(['channel_number=1', 'frame_type=1', "method=<Queue.DeclareOk(['consumer_count=0', 'message_count=0', 'queue=Features'])>"])>

In [5]:
def callback(ch, method, properties, body):
    print(f'Получен вектор признаков: {body}')
    features=json.loads(body)
    print(f'Признаки десериализованы: {features}')
    y_pred = regressor.predict(np.array(features).reshape(1, -1))
    print(f'Предсказанное значение: {y_pred}')
    ##
    channel.queue_declare(queue='y_pred')
    channel.basic_publish(exchange='',
                          routing_key='y_pred',
                          body=json.dumps(list(y_pred)))
    print(f'Сообщение с предсказанным {list(y_pred)} значением отправлено в очередь')    

In [6]:
# on_message_callback показывает какую функцию вызвать при получении сообщения
channel.basic_consume(queue='Features', on_message_callback=callback, auto_ack=True)
print('...Ожидание сообщений, для выхода нажмите CTRL+C')

...Ожидание сообщений, для выхода нажмите CTRL+C


In [ ]:
#Запустим чтение очереди. Скрипт будет работать до принудительной остановки: так мы не пропустим ни одного сообщения.
channel.start_consuming()

Получен вектор признаков: b'[0.0417084448844436, 0.0506801187398187, 0.0196615356373334, 0.0597439326260547, -0.00569681839481472, -0.00256647127337676, -0.0286742944356786, -0.00259226199818282, 0.0311929907028023, 0.00720651632920303]'
Признаки десериализованы: [0.0417084448844436, 0.0506801187398187, 0.0196615356373334, 0.0597439326260547, -0.00569681839481472, -0.00256647127337676, -0.0286742944356786, -0.00259226199818282, 0.0311929907028023, 0.00720651632920303]
Предсказанное значение: [193.01802803]
Сообщение с предсказанным [193.0180280269608] значением отправлено в очередь
Получен вектор признаков: b'[0.0526060602375023, -0.044641636506989, -0.0212953231701409, -0.0745280244296595, -0.040095639849843, -0.0376390989938044, -0.00658446761115617, -0.0394933828740919, -0.000609254186102297, -0.0549250873933176]'
Признаки десериализованы: [0.0526060602375023, -0.044641636506989, -0.0212953231701409, -0.0745280244296595, -0.040095639849843, -0.0376390989938044, -0.00658446761115617,